## Cloud masking with Sentinel 2

Cloud and cloud shadow masking of Sentinel 2 images in Python. Refactored from javascipt taken from this thread: [Sentinel 2 cloud masking](https://groups.google.com/forum/#!searchin/google-earth-engine-developers/cloud$20masking%7Csort:relevance/google-earth-engine-developers/i63DS-Dg8Sg/Kc0knF9BBgAJ)

In [46]:
from IPython.display import display, Image
import math
import ee
import os
import sys
from Py6S import *
import datetime
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric
#from cloudmasker import *
ee.Initialize()

In [47]:
def rescale(img, thresholds):
    """
    Linear stretch of image between two threshold values.
    """
    return img.subtract(thresholds[0]).divide(thresholds[1] - thresholds[0])


def ESAcloudMask(img):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
    parsed by Nick Clinton
    """
    qa = img.select('QA60')
    # bits 10 and 11 are clouds and cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)
    # both flags set to zero indicates clear conditions.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    # clouds is not clear
    cloud = clear.Not().rename(['ESA_clouds'])
    # return the masked and scaled data.
    return img.addBands(cloud)  

def shadowMask(img,cloudMaskType):
    """
    Finds cloud shadows in images
    Originally by Gennadii Donchyts, adapted by Ian Housman
    """
    def potentialShadow(cloudHeight):
        """
        Finds potential shadow areas from array of cloud heights
        returns an image stack (i.e. list of images) 
        """
        cloudHeight = ee.Number(cloudHeight)
        # shadow vector length
        shadowVector = zenith.tan().multiply(cloudHeight)
        # x and y components of shadow vector length
        x = azimuth.cos().multiply(shadowVector).divide(nominalScale).round()
        y = azimuth.sin().multiply(shadowVector).divide(nominalScale).round()
        # affine translation of clouds
        cloudShift = cloudMask.changeProj(cloudMask.projection(), cloudMask.projection().translate(x, y)) # could incorporate shadow stretch?        
        return cloudShift
    # select a cloud mask
    cloudMask = img.select(cloudMaskType)
    # make sure it is binary (i.e. apply threshold to cloud score)
    cloudScoreThreshold = 0.5
    cloudMask = cloudMask.gt(cloudScoreThreshold)
    # solar geometry (radians)
    azimuth = ee.Number(img.get('solar_azimuth')).multiply(math.pi).divide(180.0).add(ee.Number(0.5).multiply(math.pi))
    zenith  = ee.Number(0.5).multiply(math.pi ).subtract(ee.Number(img.get('solar_zenith')).multiply(math.pi).divide(180.0))
    # find potential shadow areas based on cloud and solar geometry
    nominalScale = cloudMask.projection().nominalScale()
    cloudHeights = ee.List.sequence(500,4000,500)        
    potentialShadowStack = cloudHeights.map(potentialShadow)
    potentialShadow = ee.ImageCollection.fromImages(potentialShadowStack).max()
    # shadows are not clouds
    potentialShadow = potentialShadow.And(cloudMask.Not())
    # (modified) dark pixel detection 
    darkPixels = toa.normalizedDifference(['green', 'swir2']).gt(0.25)
    # shadows are dark
    shadows = potentialShadow.And(darkPixels).rename(['shadows'])
    # might be scope for one last check here. Dark surfaces (e.g. water, basalt, etc.) cause shadow commission errors.
    # perhaps using a NDWI (e.g. green and nir)
    return img.addBands(shadows)



def quicklook(bandNames, mn, mx, region, gamma=False, title=False):
    """
    Displays images in notebook
    """
    if title:
        print('\n',title)
    if not gamma:
        gamma = 1
    visual = Image(url=toa.select(bandNames).getThumbUrl({
                'region':region,
                'min':mn,
                'max':mx,
                'gamma':gamma,
                'title':title
                }))
    display(visual)

### time and place
Define the time and place that you are looking for.

In [51]:
# region of interest
geom = ee.Geometry.Point(-155.0844, 19.7189)

# start and end of time series
startDate = ee.Date('1980-01-01')
stopDate  = ee.Date('2020-01-01')

### an image
The following code will grab the image collection between those dates, or the first in the time-series.

In [52]:
# image collection
S2col = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(geom)\
    .filterDate(startDate,stopDate)
    
# single image
S2 = ee.Image(S2col.first())# The first Sentinel 2 image

# top of atmosphere reflectance
toa = S2.divide(10000)

#### Detail some extra information

In [57]:
# Metadata
info = S2.getInfo()['properties']
scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']
# Atmospheric constituents
h2o = Atmospheric.water(geom,date).getInfo()
o3 = Atmospheric.ozone(geom,date).getInfo()
aot = Atmospheric.aerosol(geom,date).getInfo()
# Target Altitude
SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
km = alt/1000 # i.e. Py6S uses units of kilometers

NameError: name 'date' is not defined

#### Create the 6S object

In [56]:
# Instantiate
s = SixS()

# Atmospheric constituents
s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
s.aero_profile = AeroProfile.Continental
s.aot550 = aot

# Earth-Sun-satellite geometry
s.geometry = Geometry.User()
s.geometry.view_z = 0               # always NADIR (I think..)
s.geometry.solar_z = solar_z        # solar zenith angle
s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
s.altitudes.set_sensor_satellite_level()
s.altitudes.set_target_custom_altitude(km)

NameError: name 'h2o' is not defined

In [ ]:
def shadowMask(img,cloudMaskType):
    """
    Finds cloud shadows in images
    
    Originally by Gennadii Donchyts, adapted by Ian Housman
    """
    
    def potentialShadow(cloudHeight):
        """
        Finds potential shadow areas from array of cloud heights
        
        returns an image stack (i.e. list of images) 
        """
        cloudHeight = ee.Number(cloudHeight)
        
        # shadow vector length
        shadowVector = zenith.tan().multiply(cloudHeight)
        
        # x and y components of shadow vector length
        x = azimuth.cos().multiply(shadowVector).divide(nominalScale).round()
        y = azimuth.sin().multiply(shadowVector).divide(nominalScale).round()
        
        # affine translation of clouds
        cloudShift = cloudMask.changeProj(cloudMask.projection(), cloudMask.projection().translate(x, y)) # could incorporate shadow stretch?
        
        return cloudShift
  
    # select a cloud mask
    cloudMask = img.select(cloudMaskType)
    
    # make sure it is binary (i.e. apply threshold to cloud score)
    cloudScoreThreshold = 0.5
    cloudMask = cloudMask.gt(cloudScoreThreshold)

    # solar geometry (radians)
    azimuth = ee.Number(img.get('solar_azimuth')).multiply(math.pi).divide(180.0).add(ee.Number(0.5).multiply(math.pi))
    zenith  = ee.Number(0.5).multiply(math.pi ).subtract(ee.Number(img.get('solar_zenith')).multiply(math.pi).divide(180.0))

    # find potential shadow areas based on cloud and solar geometry
    nominalScale = cloudMask.projection().nominalScale()
    cloudHeights = ee.List.sequence(500,4000,500)        
    potentialShadowStack = cloudHeights.map(potentialShadow)
    potentialShadow = ee.ImageCollection.fromImages(potentialShadowStack).max()

    # shadows are not clouds
    potentialShadow = potentialShadow.And(cloudMask.Not())

    # (modified) dark pixel detection 
    darkPixels = toa.normalizedDifference(['green', 'swir2']).gt(0.25)

    # shadows are dark
    shadows = potentialShadow.And(darkPixels).rename(['shadows'])
    
    # might be scope for one last check here. Dark surfaces (e.g. water, basalt, etc.) cause shadow commission errors.
    # perhaps using a NDWI (e.g. green and nir)

    return img.addBands(shadows)

In [53]:
# top of atmosphere reflectance
toa = img.select(['B1','B2','B3','B4','B6','B8A','B9','B10', 'B11','B12'],\
                 ['aerosol', 'blue', 'green', 'red', 'red2','red4','h2o', 'cirrus','swir1', 'swir2'])\
                 .divide(10000).addBands(img.select('QA60'))\
                 .set('solar_azimuth',img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))\
                 .set('solar_zenith',img.get('MEAN_SOLAR_ZENITH_ANGLE'))

In [54]:
# clouds
toa = sentinelCloudScore(toa)
toa = ESAcloudMask(toa)

NameError: name 'ee' is not defined

In [ ]:
# cloud shadow
toa = shadowMask(toa,'cloudScore')

In [ ]:
# display region
region = geom.buffer(10000).bounds().getInfo()['coordinates']

In [ ]:
# quicklooks
quicklook(['red','green','blue'], 0, 0.25, region, gamma=1.5, title='RGB')
quicklook('cloudScore', 0, 1, region, title='Cloud Score')
quicklook('ESA_clouds', 0, 1, region, title = 'ESA Clouds (QA60)')
quicklook('shadows', 0, 1, region, title = 'Shadow mask')